<a href="https://colab.research.google.com/github/dhtrg/data_mining/blob/main/Result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pyod
from pyod.models.knn import KNN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
warnings.filterwarnings("ignore", lineno=0)
# warnings.filterwarnings("ignore", lineno=451) # IsolationForest warning
# warnings.filterwarnings("ignore", lineno=17)  # regex warning
# warnings.filterwarnings("ignore", lineno=16)  # regex warning
# warnings.filterwarnings("ignore", lineno=4913) # caveat warning for copy of slice

import time

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold,cross_val_score
from sklearn.preprocessing import StandardScaler # may not need if used with vectorized data
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import _tree
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import IsolationForest
from sklearn.neural_network import MLPClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import adjusted_rand_score, completeness_score, v_measure_score

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download(['stopwords','punkt'])
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('opinion_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [ ]:
# to optimize display of dataframe in run window
desired_width = 500
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 50)

In [ ]:
# produce csv output
def saveOut(df, textIn):
  ''' function creates a csv file for output
  parameter: single dataframe, file name
  return: notification of saved file + .csv file saved to directory  '''
  fileName = textIn
  df.to_csv(fileName, index=False)
  print(f"file {fileName} save complete")

In [ ]:
# inspect and describe
def inspect_data(data):
  # summary of dataset inspection
  stats = data.describe()
  r, col = data.shape
  check_error = data.isna().sum()
  check_missing = data.isnull().sum()
  checks = check_error.append(check_missing)
  # print(checks)
  problems = 0
  for c in checks:
    if c != 0:
      problems += 1
  all_feats = [x for x in data.columns]
  use1_feats = all_feats[5:]
  print(f"Our Sentiment Analysis dataset from Yelp! consists of {r} instances with {col} features.\n")
  print(f"There are {problems} missing values")
  print(f"In addition to {all_feats[4]}, we will keep the following features for our analysis:\n{use1_feats}\n{'*'*90}")
  return stats, use1_feats

In [ ]:
# create average columns
def expand_dataset(data):
  all_feats = [x for x in data.columns]
  use2_feats = all_feats[4:]
  snapshot_df = pd.DataFrame(data[use2_feats])
  valenceAvg = [round((r1+r2)/2) for r1, r2 in zip(snapshot_df["Rater 1 - Valence"], snapshot_df["Rater 2 - Valence"])]
  arousalAvg = [round((r1+r2)/2) for r1, r2 in zip(snapshot_df["Rater 1 - Arousal"], snapshot_df["Rater 2 - Arousal"])]
  snapshot_df.insert(5,"Average - Valence", valenceAvg)
  snapshot_df.insert(6,"Average - Arousal", arousalAvg)
  reference_numbering = [f"ref_{i:04}" for i in data["No."]]
  snapshot_df.insert(0,"ReferenceOrder",reference_numbering)
  # print(f"\nrevised dataset:\n{snapshot_df}")
  return snapshot_df

In [ ]:
def report_means(data):
  stats = data.describe()
  all_feats = [x for x in data.columns]
  use3_feats = all_feats[2:]
  index = 0
  category = []
  meanScore = []
  stdDev = []
  modeScore = []
  for i in use3_feats:
    feat = use3_feats[index]
    category.append(feat)
    meanScore.append(stats[feat][1])
    stdDev.append(stats[feat][2])
    modeScore.append(data[feat].mode()[0])
    index += 1
  rater_stats = {"Category":category,"ModeScore":modeScore, "MeanScore":meanScore, "StdDev":stdDev}
  raterStats_df = pd.DataFrame(rater_stats)
  pd.options.display.float_format = '{:.3f}'.format
  print(f"\n{raterStats_df}\n{'*'*90}")
  return raterStats_df

###### In consideration of reducing the number of class labels for all models, the following scale will be used:


*   negative [1-2]
*   somewhat negative [3-4]
*   neutral [5]
*   somewhat positive [6-7]
*   positive [8-9]



In [ ]:
# to reduce class to categorical bins
def relabel(df):
  newClass_df = pd.DataFrame(df)
  all_feats = [x for x in df.columns]
  use4_feats = all_feats[2:]

  def inner_build(feature):
    convertedScore = []
    for rank in df[feature]:
      if rank < 3:
        convertedScore.append("negative")
      elif rank < 5:
        convertedScore.append("somewhat negative")
      elif rank == 5:
        convertedScore.append("neutral")
      elif rank < 8:
        convertedScore.append("somewhat positive")
      else:
        convertedScore.append("positive")
    return convertedScore

  for feat in use4_feats:
    cs = inner_build(feat)
    feat_labeled = f"X_{feat}"
    newClass_df[feat_labeled] = cs

  temp1 = df.iloc[:,:2]
  temp2 = newClass_df.iloc[:,8:]
  newRatings_df = pd.concat([temp1,temp2], axis=1, join='inner')

  # print(newRatings_df)
  return newRatings_df

In [ ]:
def clean_all_text(df):
  ''' preprocessing function uses regex to clean text for vectorizing
  parameter: original dataframe
  return: dataframe with stripped down text in "Review text" column '''
  df_new = df.copy(deep=True)
  def reduce_lengthening(x_text):
    # to reduce extra letters that are deliberate mispellings for emphasis
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", x_text)
  def clean_URLs(x_text):
    return re.sub('((www.[^s]+)|(https?://[^s]+)|([^s]+\.com[^s]+)|([A-Za-z0-9_]+\.[A-Za-z0-9_]))','',x_text)
  def clean_hyphen(x_text):
    pattern = re.compile(r"-")
    return pattern.sub(r"", x_text)

  df_new['Review text'] = df_new['Review text'].apply(lambda x: reduce_lengthening(x))
  df_new['Review text'] = df_new['Review text'].apply(lambda x: clean_URLs(x))
  df_new['Review text'] = df_new['Review text'].apply(lambda x: clean_hyphen(x))
  df_new['Review text'] = df_new['Review text'].astype(str).str.replace(r'([\'|\*|+])','')
  df_new['Review text'] = df_new['Review text'].astype(str).str.replace(r'([\.\.|\.])',' ')
  df_new['Review text'] = df_new['Review text'].astype(str).str.replace(r'/',' ')

  return df_new

In [ ]:
def word_exclusion(df):
  ''' preprocessing function uses nltk to tokenize text for vectorizing
  parameter: cleaned dataframe
  return: dataframe with essential text in "Review text" column '''
  # filter stop words and punctuation
  def extract_words(x_text):
    stop_words = set(stopwords.words('english')) - {'not'}
    word_tokens = word_tokenize(x_text)
    filtered_text = [w.lower().strip() for w in word_tokens if not (w.strip() == '' or w in stop_words or w  in [',','.','|','~']) and not (len(w) == 1 and isinstance(w,str))]
    return filtered_text
  def lemmatizer_on_text(x_text):
    # replaces misspelled word with closest approximation from the WordNet corpus
    lm = WordNetLemmatizer()
    lemmed_text = "".join(lm.lemmatize(w)+" " for w in x_text)
    return lemmed_text

  df['Tokenized Review text']= df['Review text'].apply(lambda x: extract_words(x))
  df['Review text'] = df['Tokenized Review text'].apply(lambda x: lemmatizer_on_text(x))
  temp = df.pop('Tokenized Review text')
  df.insert(2, 'Tokenized Review text', temp)
  return df


In [ ]:
# to create a bag of words that results in vector values
def bag_of_vectors(df):
  '''preprocessing function uses sklearn to normalize tokenized text based on frequencies in the document
  parameter: tokenized dataframe
  return: df_bow -- features are individual words found in text, values are vectors
        : vectorBow_df -- df_bow with reference features and original text blob appended (not currently used) '''
  bagStart_time = time.time()
  print(f"{'@@@'} Start bagging process...")
  this_df = pd.DataFrame(df)
  vectorizer = TfidfVectorizer(stop_words='english', strip_accents='unicode')
  X = vectorizer.fit_transform([review for review in this_df['Review text']])
  df_bow = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
  #drop columns that contain numeric values
  for col in df_bow.columns:
    if re.search(r'\d', col):
      df_bow.drop(columns = col, inplace=True)
  vectorBow_df = pd.concat([this_df,df_bow], axis=1, join='inner')
  bagEnd_time = time.time()
  delta_time = bagEnd_time - bagStart_time
  print(f"... Bagging completed in {delta_time:.4f} seconds {'@@@'}")
  return vectorBow_df, df_bow

In [ ]:
# recode dataset for boolean options if a word is present in the document / instance value = 1, else = 0
def transaction_set(df_all, df_matrix):
  encodeStart_time = time.time()
  print(f"{'@@@'} Start encoding process...")
  df_transaction = df_all.copy(deep=True)
  allFeats = [x for x in df_transaction.columns]
  useFeats = allFeats[3:9]
  # print(useFeats)

  df_bools = df_matrix.copy(deep=True)
  df_bools[df_bools > 0] = 1

  for name in useFeats:
    df_bools[name] = df_transaction[name]

  encodeEnd_time = time.time()
  delta_time = encodeEnd_time - encodeStart_time
  print(f"... Encoding completed in {delta_time:.4f} seconds {'@@@'}")
  # print(df_bools)

  return df_bools


In [ ]:
def validate_model(clf_name,classifier,df,target_df,k_fold):
  """this function implement k-fold cross-validation for classifier
  params: clf_name: name of the classifier
          classifier: classifier of interest
          df: dataframe of independent attributes
          target_df: a dataframe containing all targets of interest
          k_fold: number of folds for cross-validation
  return: a dataframe with 'precision_macro','recall_macro', 'f1_macro','accuracy' for the target columns
  """
  report_dict = {}
  metrics = ['precision_macro','recall_macro', 'f1_macro','accuracy']
  print(f'{clf_name} classifier {k_fold}-fold cross-validation for all targets')
  for target in target_df.columns:
    scores = []

    target_col = target_df[target]
    for metric in metrics:
      scores.append(np.mean(cross_val_score(classifier, df ,target_col, cv=k_fold,scoring = metric)))
    report_dict[target]= scores
  report_metric = pd.DataFrame(report_dict,index=metrics)
  print(report_metric)
  return report_metric

In [ ]:
# Decision Tree classifier
def decision_tree(df_textMatrix, target_df):
  beginDT = time.time()
  print(f"\n{'*'*90}\nModeling Decision Tree ...\n")
  dcs_tree = DecisionTreeClassifier(max_depth=10, random_state=1234)
  P1_DT_report_metric = validate_model('Decision Tree',dcs_tree,df_textMatrix,target_df,10)
  endDT = time.time()
  delta_time = endDT-beginDT
  print(f"... Decision Tree took {delta_time:.4f} seconds to complete\n{'*'*90}\n")
  return P1_DT_report_metric

In [ ]:
# Multinominal Naive Bayes Classifier
def bayesMN(df_textMatrix, target_df):
  beginMNB = time.time()
  print(f"\n{'*'*90}\nMulinominal Naive Bayes ...\n")
  multi_naive_clf = MultinomialNB()
  P1_MNB_report_metric = validate_model('Mulinominal Naive Bayes',multi_naive_clf,df_textMatrix,target_df,10)
  endMNB = time.time()
  delta_time = endMNB-beginMNB
  print(f"... Mulinominal Naive Bayes took {delta_time:.4f} seconds to complete\n{'*'*90}\n")
  return P1_MNB_report_metric

In [ ]:
def cluster_kmeans(matrix,target):
  # requires normalized data -- using vectorized data
  # requires target label values to be converted from nominal to values 0-4
  target_lbl = target
  target_lbl[target_lbl == "negative"] = 4
  target_lbl[target_lbl == "somewhat negative"] = 3
  target_lbl[target_lbl == "neutral"] = 2
  target_lbl[target_lbl == "positive"] = 0
  target_lbl[target_lbl == "somewhat positive"] = 1
  def validate_cluster(clf_name,classifier,df,target_df,k_fold):
    '''This function passes the k-means model to k-fold cross-validation with scoring unique to a cluster model
    param:  clf_name - descriptive text
            classifier - the model object
            df - vectorized text matrix dataframe
            target_df - class labels of 6 ratings related to the dataset
            k-fold - integer, number of folds
    return: report_metric -- a dataframe of the scores of each validated model '''
    report_dict = {}
    metrics = ['completeness_score','adjusted_rand_score', 'v_measure_score']
    print(f'{clf_name} classifier {k_fold}-fold cross-validation for all targets')
    for target in target_df.columns:
      scores = []

      target_col = target_df[target]
      for metric in metrics:
        scores.append(np.mean(cross_val_score(classifier, df ,target_col, cv=k_fold,scoring = metric)))
      report_dict[target]= scores
    report_metric = pd.DataFrame(report_dict,index=metrics)
    print(report_metric)
    return report_metric

  beginKM = time.time()
  print(f"\n{'*'*90}\nModeling K-means cluster ...\nthis process may take ~50 minutes to complete")
  km_model = KMeans(n_clusters=5, init="k-means++", n_init=10, max_iter=300, random_state=51)
  P1_KM_report_metric = validate_cluster("K-means Clustering", km_model, matrix, target_lbl, 10)

  endKM = time.time()
  delta_time = endKM-beginKM
  print(f"... K-means cluster took {delta_time:.4f} seconds to complete\n{'*'*90}\n")
  return P1_KM_report_metric

In [ ]:
# Bernoulli Naive Bayes classifier
def bayesBernoulli(df_textMatrix, target_df):
  beginBNB = time.time()
  bernoulli_naive_clf = BernoulliNB()
  P3_CB_report_metric = validate_model('Bernoulli Naive Bayes',bernoulli_naive_clf,df_textMatrix,target_df,10)
  endBNB = time.time()
  delta_time = endBNB-beginBNB
  print(f"... BernoulliNaive Bayes took {delta_time:.4f} seconds to complete\n{'*'*90}\n")

  return P3_CB_report_metric

In [ ]:
def ann_classifier(matrix,target):
  # requires normalized data -- using vectorized data
  def validate_ann(clf_name,classifier,df,target_df,k_fold):
    '''This function passes the k-means model to k-fold cross-validation with scoring unique to a cluster model
    param:  clf_name - descriptive text
            classifier - the model object
            df - vectorized text matrix dataframe
            target_df - class labels of 6 ratings related to the dataset
            k-fold - integer, number of folds
    return: report_metric -- a dataframe of the scores of each validated model '''
    report_dict = {}
    metrics = ['precision_macro','recall_macro', 'f1_macro','accuracy']
    print(f'{clf_name} classifier {k_fold}-fold cross-validation for all targets')
    for target in target_df.columns:
      scores = []

      target_col = target_df[target]
      le = LabelEncoder()
      target_col = le.fit_transform(target_col)
      for metric in metrics:
        scores.append(np.mean(cross_val_score(classifier, df ,target_col, cv=k_fold,scoring = metric)))
      report_dict[target]= scores
    report_metric = pd.DataFrame(report_dict,index=metrics)
    print(report_metric)
    return report_metric
  beginANN = time.time()
  print(f"\n{'*'*90}\nArtificial Neural Network ...\nthis process may take ~1 hour to complete")
  target_lbl = target
  ANN_model = MLPClassifier(hidden_layer_sizes=(50,50), learning_rate='adaptive', random_state=51)
  P3_ANN_report_metric = validate_ann("Artificial Neural Network", ANN_model, matrix, target_lbl, 5)
  endANN = time.time()
  delta_time = endANN-beginANN
  print(f"... Artificial Neural Network took {delta_time:.4f} seconds to complete\n{'*'*90}\n")
  return P3_ANN_report_metric


In [ ]:
def rules_based(df,target):
  """ This function extracts and print out rules from a decision tree
  param: df: a dataframe of independent features
         target: name of the target column
  return: display a dataframe of rules including antecedent, consequent, accuracy and coverage of each rule
  """
  rules_filename = f"{target}.csv"
  beginRules = time.time()
  print(f"\n{'*'*90}\nModeling with Rule-based ...")


  def get_rules(tree, feature_names, class_col):
    """ This function extracts rules from a decision tree
    param: tree: a tree object
          feature_names: names of the independent attributes
          class_col: the target column
    return: a dataframe of rules including antecedent, consequent, accuracy and coverage of each rule
    """
    tree_ = tree.tree_
    feature_name = [
          feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
          for i in tree_.feature
      ]

    paths = []
    path = []
    ante = []
    conse = []
    accuracy = []
    cover = []
    def recurse(node, path, paths):
      """ This function recusively traverse the decision tree, store data of each node and decision path
      param: node: node in the tree object
            path: path to the target
            paths: a list that stores all paths
      """

      if tree_.feature[node] != _tree.TREE_UNDEFINED:
          name = feature_name[node]
          threshold = tree_.threshold[node]
          p1, p2 = list(path), list(path)
          p1 += [f"({name} <= {np.round(threshold, 3)})"]
          recurse(tree_.children_left[node], p1, paths)
          p2 += [f"({name} > {np.round(threshold, 3)})"]
          recurse(tree_.children_right[node], p2, paths)
      else:
          path += [(tree_.value[node], tree_.n_node_samples[node])]
          paths += [path]

    recurse(0, path, paths)

    # sort paths by samples count of each class label in each path
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]

    #store antecedent, consequent, accuracy and coverage of each rule in seperate lists
    for path in paths:
      rule = []
      for p in path[:-1]:
        if "<=" not in p:
          rule.append(str(p))
      ante.append(rule)
      classes = path[-1][0][0]

      l = np.argmax(classes)
      conse.append(class_col[l])
      accuracy.append(np.round(classes[l]/np.sum(classes),2))
      cover.append(np.round(path[-1][1]/2000,2))

    #create a dataframe of rules including antecedent, consequent, accuracy and coverage of each rule
    rules = pd.DataFrame({'Antecedent':ante,'Consequent':conse,'Accuracy':accuracy,'Coverage':cover})

    return rules


  # Prepare the data data

  X = df.iloc[:,:-6]
  y = df[target]

  # Fit the regressor, set max_depth = 3
  dcs_tree = DecisionTreeClassifier(max_depth=10, random_state=1234)
  model = dcs_tree.fit(X, y)
  # Print rules
  rules = get_rules(dcs_tree, X.columns, y)
  DF_rules = pd.DataFrame(rules)
  print(rules)
  saveOut(DF_rules, rules_filename)
  endRules = time.time()
  delta_time = endRules-beginRules
  print(f"... Rule-based took {delta_time:.4f} seconds to complete\n{'*'*90}\n")



In [ ]:
def isolation_forest(df_o, df_matrix):
  ''' This function detects whether an instance is an outlier and creates a csv file containing anomoulous instances
  param: df -- a dataframe of only the vectorized values related to the "Review text"
  return: if_outliers -- a dataframe containing instances of outliers/anomalies with their scores, a csv file of those instances, and a graph
  '''
  beginIF = time.time()
  print(f"\n{'*'*90}\nModeling Isolation Forest ...")
  df_transaction = df_matrix.copy(deep=True)
  df_o = pd.DataFrame(df_o)
  X_train, X_test = train_test_split(df_transaction, test_size=0.4,random_state=51)
  # create isolation forest model and fit training data
  iForest = IsolationForest(n_estimators=500,
                            contamination=0.02,
                            max_features=1000,
                            random_state=51)
  iForest.fit(X_train)

  # store labels of predicted anomalies (1= inlier, -1= oulier)
  if_labels_train = iForest.predict(X_train)
  if_labels_test = iForest.predict(X_test)

  # store anomaly scores
  if_score_train = iForest.decision_function(X_train)
  if_score_test = iForest.decision_function(X_test)

  # stitch the labels and scores back together
  X_train['outlierLabel'] = if_labels_train
  X_test['outlierLabel'] = if_labels_test
  X_train['outlierScore'] = if_score_train
  X_test['outlierScore'] = if_score_test
  rezipXframes = pd.concat([X_train, X_test])

  # apply anomaly classification to the original dataset
  df_o['outlierScore'] = rezipXframes['outlierScore']
  df_o['outlierLabel'] = rezipXframes['outlierLabel']
  labelTemp = df_o['outlierLabel']
  df_o['is_outlier'] = ["yes" if x==-1 else "no" for x in labelTemp]

  plt.scatter(df_o['outlierScore'],df_o['No.'], c=df_o['outlierLabel'], cmap='winter')
  plt.title("Scatter Plot of Datapoints by IF Outlier Score")
  plt.xlabel("IF Outlier Score (outlier << 0)")
  plt.ylabel("Reference No.")
  plt.show()

  # create dataframe of anomalies only
  if_outliers = df_o.loc[df_o['is_outlier'] == "yes"]
  if_outliers.drop(axis=1, columns="outlierLabel", inplace=True)

  print(f"Created a dataframe containing the {if_outliers.shape[0]} items that were identified as outliers/anomalies from the dataset via Isolation Forest Model")
  # create csv of anomalies only
  # saveOut(if_outliers, "Isolation_Forest_outliers.csv")

  endIF = time.time()
  delta_time = endIF-beginIF
  print(f"... Isolation Forsest took {delta_time:.4f} seconds to complete\n{'*'*90}\n")

  return if_outliers

In [ ]:
def knn_anomaly_detection(df_o, df_matrix):
  ''' This function detects whether an instance is outlier and create a csv file containing anomoulous instances
  param: vectorized_df: a dataframe of vectorized text
  return: knn_outliers -- a dataframe containing outliers and create a csv file containing anomoulous instances
  '''
  beginKNN = time.time()
  print(f"\n{'*'*90}\nModeling K-nearest Neighbor ...")
  vectorized_df = df_matrix.copy(deep=True)
  df_o = pd.DataFrame(df_o)
  #create the knn models
  mdl = KNN(n_neighbors=50, contamination=0.02)
  mdl.fit(vectorized_df)

  # Outlier scores
  scores = mdl.decision_scores_
  df_o['Outlier score'] = scores

  #label of the records: 1 means outliers, 0 means not outliers
  labels = mdl.labels_
  df_o['is_outlier'] = labels

  plt.scatter(df_o['Outlier score'],df_o['No.'],c= df_o['is_outlier'])
  plt.title("Scatter Plot of Datapoints by KNN Outlier Score")
  plt.xlabel("KNN Outlier Score (outlier >> 1)")
  plt.ylabel("Reference No.")
  plt.show()

  #change the 'is_outlier' value from 1 to 'yes' and 0 to 'no'
  df_o['is_outlier'].replace(1,'yes',inplace = True)
  df_o['is_outlier'].replace(0,'no',inplace = True)
  knn_outliers=df_o.loc[df_o["is_outlier"]=='yes']
  print(f"Created a dataframe containing the {knn_outliers.shape[0]} objects that were identified as outliers/anomalies from the dataset via KNN Model")
  # saveOut(knn_outliers, "KNN_outliers.csv")

  endKNN = time.time()
  delta_time = endKNN-beginKNN
  print(f"... K-nearest Neighbor took {delta_time:.4f} seconds to complete\n{'*'*90}\n")


  return knn_outliers

In [ ]:
# execute file
def main():
  original_data = pd.read_csv(r'https://raw.githubusercontent.com/farberta2022/DSC411_SP22_final/main/sampleReviewData_030822.csv')
  # *** preprocessing standard***
  inspect_data(original_data)
  # returns revised dataset with reference to original, columns 4-8, and the two new average columns (all instances present)
  revisedMain_df = expand_dataset(original_data)
  # *** preprocessing special analysis***
  print(f"Statistical description of original dataset")
  raterStats_df = report_means(revisedMain_df)
  # # returns revised dataset of reduced class labels targets
  # df_w5rank has 8 columns - ref#, Review text, and six 5-rank columns
  df_w5rank = relabel(revisedMain_df)
  # # passes "review text" through text mining preprocessing steps
  df_newText = clean_all_text(df_w5rank)
  # df_newText2 has 9 columns - ref#, Review text, Tokenized Text, and six 5-rank columns
  df_newText2 = word_exclusion(df_newText)
  # df_baggedText includes 9 columns from above plus df_textMatrix of 10k+ columns of vectorized values
  df_baggedText, df_textMatrix = bag_of_vectors(df_newText2)
  # df_textEncoded includes df_textMatrix of 10k+ columns of boolean values + six  5-rank columns
  df_textEncoded = transaction_set(df_baggedText, df_textMatrix)
  #this datafrane store the target columns
  target_df = df_textEncoded.iloc[:,-6:]

  # # # Project 1: Classification Basics
  # print(f"\n{'*'*90}\n{'*'*40}Project  1{'*'*40}")
  # # Decision Tree classifier                                          **** tested, works
  # P1_report1 = decision_tree(df_textMatrix, target_df)

  # # Multinominal Naive Bayes Classifier since this would be a better choice than Decesion tree
  # #                                                                  **** tested, works
  # P1_report2 = bayesMN(df_textMatrix, target_df)

  # # K-means cluster
  # # TODO: need to test in this program, tests in dedicated program worked fine
  # P1_report3 = cluster_kmeans(df_textMatrix,target_df)


  # # # # # Project 2: detect outliers with ioslation forest algorithm
  # # #TODO: check output for uniformity
  # print(f"\n{'*'*90}\n{'*'*40}Project  2{'*'*40}")
  # # # # Project 2: detect outliers with ioslation forest algorithm
  # #                                                                  **** tested, works
  # outlier_iForest = isolation_forest(original_data, df_textMatrix)
  # # # # Project 2: detect outliers with knn-algorithm
  # #                                                                  **** tested, works
  # knn_anomaly_detection(original_data, df_textMatrix)
  ##****** currently have saveOutput commented out in the anomaly detection functions*****

  # # # # Project 3: Classification Advanced
  # print(f"\n{'*'*90}\n{'*'*40}Project  3{'*'*40}")
  # #Bernoulli Naive Bayes classifier, which only takes into consideration the presence of a word
  # #                                                                  **** tested, works
  # P3_report1 = bayesBernoulli(df_textMatrix, target_df)

  #rules-based classifier                                          **** tested, works
  for target in target_df.columns:
    print(f'Rules-based mining for {target}')
    rules_based(df_textEncoded,target)

  ## Artificial Neural Network
  # P3_report3 = ann_classifier(df_textMatrix,target_df)

  # saveOut(df_baggedText, "df_withBaggedText.csv")
  # saveOut(df_textMatrix, "vect_textMatrix.csv")
  # saveOut(df_textEncoded, "bool_textMatrix.csv")
  # print(f"test code:\t{type(feats_subset1)}\n{feats_subset1}")
  # print(f"\ntest\n{df_textEncoded}")



  pass

In [ ]:
if __name__=="__main__":
  main()

Our Sentiment Analysis dataset from Yelp! consists of 2000 instances with 9 features.

There are 0 missing values
In addition to Review text, we will keep the following features for our analysis:
['Rater 1 - Valence', 'Rater 1 - Arousal', 'Rater 2 - Valence', 'Rater 2 - Arousal']
******************************************************************************************
Statistical description of original dataset

            Category  ModeScore  MeanScore  StdDev
0  Rater 1 - Valence          9      7.064   2.104
1  Rater 1 - Arousal          7      6.085   2.155
2  Rater 2 - Valence          8      6.870   2.099
3  Rater 2 - Arousal          7      6.193   2.129
4  Average - Valence          8      6.905   1.986
5  Average - Arousal          8      6.136   2.061
******************************************************************************************
@@@ Start bagging process...
... Bagging completed in 25.8852 seconds @@@
@@@ Start encoding process...
... Encoding completed in 0.993